#Initialize tensorflow-GPU

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip install tensorflow-gpu
import tensorflow as tf 
#test111

In [0]:

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at : {}'.format(device_name))  

Found GPU at : /device:GPU:0


#Data Preprocessing

##importing Libraries and Data

In [0]:
#from google.colab import files
#file = files.upload()
import numpy as np
import pandas as pd
dataset = pd.read_csv("/content/gdrive/My Drive/AIProj/pickups_2019.csv")
dataset = dataset.drop(['cluster_id'], axis = 1)             
dataset.head()

In [0]:
dataset = pd.read_csv("/content/gdrive/My Drive/AIProj/pickups_2019.csv")
dataset['weekday']

##Getting the dependent and independent variables

In [0]:
def ToMinutes(t):
  (h, m, s) = t.split(':')
  result = float(h) *60+ float(m) + float(s)/60
  return str(result)

In [0]:
timeCol = dataset['time']
dataset['time'] = timeCol.apply((lambda x: ToMinutes(x)))
X = dataset.iloc[:,1:-1].values 
y = dataset['pickups'].values


In [0]:
MAX_PICKUP_BIKES = y.max() +1
NUM_OF_INPUTS = X.shape[1]

##Splitting the dataset into training set and Test set 

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)


##Feature scaling

In [0]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_Y = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


In [0]:
FIRST_LAYAR = 200
SECONED_LAYAR = 200
OUTPUT_ACTIVATION_FUNC = 'softmax'
OPTIMIZER = 'rmsprop' #'adam' #stochastich gradient descent
NUM_EPOCH = 100
BATCH_SIZE = 75

#Builed the NN

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
classifier = Sequential()

classifier.add(Dense(output_dim = FIRST_LAYAR  , init = 'uniform',activation ='relu',input_dim = NUM_OF_INPUTS ))
classifier.add(Dense(output_dim = SECONED_LAYAR, init = 'uniform', activation = 'relu' ))
classifier.add(Dense(output_dim =355, init = 'uniform', activation = OUTPUT_ACTIVATION_FUNC ))
 

#Define Optimizer and Loss function 

In [0]:
classifier.compile(optimizer = OPTIMIZER , loss = 'sparse_categorical_crossentropy',metrics = ['accuracy']) 

Set Values: 


#Train the data 

In [0]:
classifier.fit(X_train, Y_train, batch_size = BATCH_SIZE , nb_epoch = NUM_EPOCH)
from keras.models import load_model
classifier.save_weights('/content/gdrive/My Drive/AIProj/my_model_weights.h5')

#Results TODO: Check how to save wights

In [0]:
Y_pred = classifier.predict(X_test)
Y_get = []
for i in range(len(Y_pred)):
  Y_get.append(Y_pred[i].argmax())

In [0]:
total = 0 
correct = 0 
wrong = 0 
for i in range(len(Y_pred)):
  total = total +1
  if(Y_get[i] == Y_test[i]):
     correct = correct + 1
  else:
    wrong = wrong + 1
    
print("Total "+ str(total) )
print("Correct " + str(correct))
print("Wrong " + str(wrong))
print("acc:" + str(float(correct)/total) )
    

Total 30138
Correct 9819
Wrong 20319
acc:0.3258013139558033


#Save wights

In [0]:
from keras.models import load_model
classifier.save_weights('my_model_weights.h5')

#Load Wights

In [0]:
classifier.load_weights('my_model_weights.h5')

#Evaluating the ANN 

#Using K-Fold Cross Validation 


In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
  classifier = Sequential()

  classifier.add(Dense(output_dim = FIRST_LAYAR  , init = 'uniform',activation ='relu',input_dim = NUM_OF_INPUTS ))
  classifier.add(Dense(output_dim = SECONED_LAYAR, init = 'uniform', activation = 'relu' ))
  classifier.add(Dense(output_dim = 355, init = 'uniform', activation = OUTPUT_ACTIVATION_FUNC ))
  classifier.compile(optimizer = 'adam'  , loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])
  return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 75, nb_epoch = 250 )
accuracies = cross_val_score(estimator = classifier, X = X_train, y = Y_train, cv =10, n_jobs = -1)# cv num of folds , n_jobs =-1 usingall the GPU
#Accuracies is a vector with all the #folds results(in our case 10) 
#The real Result is the mean of this vector, also we need to check for hight or low variance to know if our results are good
mean = accuracies.mean()
variance = accuracies.std()
mean 
variance
  

0.005668231143304357

In [0]:
mean

0.32813215512885086

#Improving the ANN

#Droput Regularization to reduce overfitting if needed 
While Getting big variance, meaning the model was doing well on some and than really bad on others, so he is overfitting on the training.
This startagy will help fixing it. 

#Tuning The ANN


In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer,first, seconed ):
  classifier = Sequential()

  classifier.add(Dense(output_dim = first  , init = 'uniform',activation ='relu',input_dim = NUM_OF_INPUTS ))
  classifier.add(Dense(output_dim = seconed, init = 'uniform', activation = 'relu' ))
  classifier.add(Dense(output_dim = 355, init = 'uniform', activation = OUTPUT_ACTIVATION_FUNC ))
  classifier.compile(optimizer = optimizer  , loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])
  return classifier

classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32 ],'nb_epochs': [250, 500], 'optimizer' : ['adam', 'rmsprop'], 'first': [100,200], 'seconed':[100,200]}
grid_search = GridSearchCV(estimator = classifier, param_grid = parameters, scoring = 'accuracy', cv = 10)  
grid_serach = grid_search.fit(X_train, Y_train)                     
best_parameters = grid_serach.best_params_
best_accuracy = grid_search.best_score_